# Step 1:
  模型假设:
    1. 7种不同投资产品(投资期限、起投金额、到期收益、剩余可头金额)
    2. 4种不同现金优惠券
    
  注：在用户输入可承受投资期限后，如：360天。后台会自动在产品库中筛选出小于360天的投资产品。

In [3]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

In [2]:
product = pd.read_csv("/home/dzn/Project/d07_optimize/products.csv")
product = product.sort_values(by = ["rate", "duedays", "starting"], ascending=[False, True, True])
product

,product_id,rate,duedays,starting,outlanding
3,p_004,0.076,358,500000,510000
2,p_003,0.073,342,50000,2000000
6,p_007,0.073,348,50000,130000
5,p_006,0.070,358,10000,1400000
1,p_002,0.068,358,1000,1700000
4,p_005,0.066,348,100,2000000
0,p_001,0.058,182,100,1500000


In [3]:
product.index = range(7)
product

,product_id,rate,duedays,starting,outlanding
0,p_004,0.076,358,500000,510000
1,p_003,0.073,342,50000,2000000
2,p_007,0.073,348,50000,130000
3,p_006,0.070,358,10000,1400000
4,p_002,0.068,358,1000,1700000
5,p_005,0.066,348,100,2000000
6,p_001,0.058,182,100,1500000


In [4]:
coupon_loc  = '/home/dzn/Project/d07_optimize/coupons.csv'
coupon      = pd.read_csv(coupon_loc)
coupon

,coupon_id,starting,rebate,duedays
0,coupon_01,1000,20,342
1,coupon_02,3000,30,358
2,coupon_03,100,20,182
3,coupon_04,5000,50,348
4,coupon_05,10000,100,358
5,coupon_06,120,20,358


In [6]:
abc = pd.DataFrame()
type(abc)

pandas.core.frame.DataFrame

In [8]:
a = [1,2,3,4]
a[0] += 4
a

[5, 2, 3, 4]

In [12]:
coupon = pd.read_csv("/home/dzn/Project/d07_optimize/coupons.csv")
coupon

,coupon_id,starting,rebate,duedays
0,coupon_01,1000,20,342
1,coupon_02,3000,30,358
2,coupon_03,100,20,182
3,coupon_04,5000,50,348
4,coupon_05,10000,100,358
5,coupon_06,120,20,358


In [105]:
product.loc[0,:]

product_id     p_004
rate           0.076
duedays          358
starting      500000
outlanding    510000
Name: 0, dtype: object

In [89]:
#np.dot(product.rate, product.starting)

In [90]:
def dotMultiply(a,b):
    result = []
    for i in np.arange(len(a)):
        result.append(a[i]*b[i])
    return result

In [93]:
profits = dotMultiply(product.rate,product.starting)
[round(i, 2) for i in profits]

[38000.0, 3650.0, 3650.0, 700.0, 68.0, 6.6, 5.8]

In [83]:
product = product[product.duedays <= 350]
product = product[product.starting <= 10000]
product

,product_id,rate,duedays,starting,outlanding
5,p_005,0.066,348,100,2000000
6,p_001,0.058,182,100,1500000


In [7]:
# Model Info

# Step 2:
   建立模型：  
     1. x0，x1，...，x6分别对应购买prod_01，prod_02，...，prod_07金额 
     2. 建立最大收益函数：
        fun = - (x[0]*profits[0] + x[1]*profits[1] + x[2]*profits[2] + x[3]*profits[3] + x[4]*profits[4] + x[5]*profits[5] + x[6]*profits[6])

In [8]:
#def evalx(a):
#    result = a if a > 1 else 0
#    return result

In [9]:
fun = lambda x: -(x[0]*profits[0] + x[1]*profits[1] + x[2]*profits[2] + x[3]*profits[3] + x[4]*profits[4] + x[5]*profits[5] + x[6]*profits[6])  

# Step 3:
   边界条件:

       1. sum(x[0],x[1],...,x[6]) <= 输入投资额 (amount = 20000)
       2. any(x[0],x[1],...,x[6]) >= 0


In [10]:
amount = 530000

In [11]:
#func = amount - (x[0]*product.starting[0] + x[1]*product.starting[1] + x[2]*product.starting[2] + x[3]*product.starting[3] + x[4]*product.starting[4] + x[5]*product.starting[5] + x[6]*product.starting[6])   

In [12]:
cons = ({'type':'eq', 'fun':lambda x: amount - (x[0]*product.starting[0] + x[1]*product.starting[1] + x[2]*product.starting[2] + x[3]*product.starting[3] + x[4]*product.starting[4] + x[5]*product.starting[5] + x[6]*product.starting[6]) })

In [13]:
bnds = ((0, None), (0, None), (0, None), (0, 100), (0, 1000), (0, None), (0, None))

In [14]:
bnds = [(0, 1.0*product.outlanding[i]/product.starting[i])  for i in range(7)]
bnds

[(0, 1.02),
 (0, 40.0),
 (0, 2.6),
 (0, 140.0),
 (0, 1700.0),
 (0, 20000.0),
 (0, 15000.0)]

In [15]:
#test.append((0, 0))
#test

In [16]:
optimize = minimize(fun, (1, 1, 1, 1, 1, 1, 1), method='SLSQP', bounds=bnds, constraints = cons, options = {'maxiter': 25})

In [17]:
optimize

  status: 0
 success: True
    njev: 6
    nfev: 75
     fun: -40219.999999598585
       x: array([  1.02000000e+00,   2.00000001e-01,   2.00000001e-01,
         4.99092427e-10,   6.69786108e-11,   8.38000440e-12,
         1.51224998e-11])
 message: 'Optimization terminated successfully.'
     jac: array([ -3.80000000e+04,  -3.65000000e+03,  -3.65000000e+03,
        -7.00000000e+02,  -6.80000000e+01,  -6.60009766e+00,
        -5.79980469e+00,   0.00000000e+00])
     nit: 8

In [18]:
optimize.success

True

In [19]:
raw_investratio = [round(optimize.x[i], 2) for i in range(7)]
raw_investratio

[1.02, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0]

In [20]:
raw_investamount = [round(v * product.starting[i], 2) for i, v  in enumerate(optimize.x)]
raw_investamount 

[510000.0, 10000.0, 10000.0, 0.0, 0.0, 0.0, 0.0]

In [21]:
need_invest = [raw_investamount[i] for i in range(len(raw_investamount)) if raw_investratio[i] > 0 and raw_investratio[i] < 1]
need_invest =sum(need_invest)
need_invest

20000.0

In [42]:
raw_investratio = [0, 0, 0, 0, 0, 0, 0]
need_invest = 530000

In [49]:
def optimizeInvest(invest, product, raw_investratio, profits):
    
    tmp = 0
    investratio = []
    prod_cnt = len(raw_investratio)
    for i, v in enumerate(raw_investratio):
        if v >= 1:
            investratio.append(v)
        elif v > 0 and v < 1 :
            tmp = i
            break
    print(investratio)
    new_product = product.loc[range(tmp, prod_cnt),:]
    new_product.index = range(len(new_product))
    print(new_product)
    print(invest)
    t = 0
    while invest > 0 and t < 4:
        for i in range(len(new_product)):
            print(i)
            t += 1
            if invest >= new_product.starting[i] :
                if invest <= new_product.outlanding[i] :
                    investratio.append( 1.0*invest/new_product.starting[i] )
                    invest = 0
                else :
                    investratio.append( 1.0*new_product.outlanding[i]/new_product.starting[i] )
                    invest = invest - new_product.outlanding[i]
                    new_product = new_product.loc[range(i+1, len(new_product)), :]
                    new_product.index = range(len(new_product))
                break
            else :
                investratio.append(0)
                new_product = new_product.loc[range(i+1, len(new_product)), :]
                new_product.index = range(len(new_product))
                break
    #result = np.dot(profits, investratio)    
    return investratio
 

In [50]:
bbb = optimizeInvest(need_invest, product, raw_investratio, profits)

[]
  product_id   rate  duedays  starting  outlanding
0      p_004  0.076      358    500000      510000
1      p_003  0.073      342     50000     2000000
2      p_007  0.073      348     50000      130000
3      p_006  0.070      358     10000     1400000
4      p_002  0.068      358      1000     1700000
5      p_005  0.066      348       100     2000000
6      p_001  0.058      182       100     1500000
530000
0
0
0
0


In [51]:
bbb

[1.02, 0, 0, 2.0]

In [54]:
max(bbb) > 3

False

In [32]:
a = product.loc[[i for i in range(4)],:]
a

,product_id,rate,duedays,starting,outlanding
0,p_004,0.076,358,500000,510000
1,p_003,0.073,342,50000,2000000
2,p_007,0.073,348,50000,130000
3,p_006,0.070,358,10000,1400000


In [36]:
portfolio = dotMultiply(bbb, a.starting)
portfolio

[510000.0, 0, 0, 20000.0]

In [33]:
profits[:4]

[38000.0, 3649.9999999999995, 3649.9999999999995, 700.00000000000011]

In [37]:
profit = np.dot(bbb, profits[:len(bbb)])
profit

40160.0

In [66]:
a.loc[2,:]

product_id     p_007
rate           0.073
duedays          348
starting       50000
outlanding    130000
Name: 2, dtype: object

In [59]:
product.pop

<bound method DataFrame.pop of   product_id   rate  duedays  starting  outlanding
0      p_004  0.076      358    500000      510000
1      p_003  0.073      342     50000     2000000
2      p_007  0.073      348     50000      130000
3      p_006  0.070      358     10000     1400000
4      p_002  0.068      358      1000     1700000
5      p_005  0.066      348       100     2000000
6      p_001  0.058      182       100     1500000>

AttributeError: 'DataFrame' object has no attribute 'prduct_id'

In [15]:
-1*round(optimize.fun, 2)

40220.0

In [64]:
a = ['x', 'y', 'z']
b = [1, 2, 2]
c = []
for i, v in enumerate(a):
    c.append(v+"*"+str(b[i]))
    if i < 2:
        c.append("+")

In [47]:
''.join(i for i in c)

'x*1+y*2+z*2'

In [70]:
def formula(a,b):
    exp = []
    for i, v in enumerate(a):
        exp.append(v+"*"+str(b[i]))
        if i < len(a) -1 :
            exp.append("+")
    return ' '.join(i for i in exp)

In [71]:
formula(a, b)

'x*1 + y*2 + z*2'

In [206]:
a = pd.read_csv("a.csv")

In [207]:
a.head()

,mobile_3,cnt
0,130,1
1,130,1
2,130,1
3,130,1
4,130,1


In [208]:
b = a.cnt.groupby(a.mobile_3)

In [210]:
b.count()

mobile_3
130    9589
131    9105
132    8216
133     700
134    2629
135    1302
136    1079
137    2615
138      91
139      51
147     153
150     983
151     974
152    1254
153     441
155    4409
156    6683
157     845
158    1533
159    1545
170    1255
171      98
173       1
175       1
177      24
178     256
180     323
181      74
182     642
183     668
184     927
185      68
186    1297
187     718
188     986
189     217
dtype: int64

In [219]:
import random
random.seed(1)

In [224]:
list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
slice = random.sample(list, 5)
slice

[3, 9, 8, 1, 7]

In [225]:
product

,product_id,rate,duedays,starting,outlanding
0,p_001,0.058,182,100,1500000
1,p_005,0.066,348,100,2000000
2,p_002,0.068,358,1000,1700000
3,p_006,0.070,358,10000,1400000
4,p_007,0.073,348,50000,130000
5,p_003,0.073,342,50000,2000000
6,p_004,0.076,358,500000,510000


In [228]:
product.loc[slice,:]

,product_id,rate,duedays,starting,outlanding
3,p_006,0.070,358,10000,1400000
9,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
1,p_005,0.066,348,100,2000000
7,NaN,NaN,NaN,NaN,NaN
